In [1]:
import numpy as np
import pandas as pd
import keras
import warnings
warnings.filterwarnings("ignore")

/home/abhinav/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df= pd.read_csv("/home/abhinav/Documents/dataset/data.csv", header=None, skiprows=1)
df.head()

,0,1
0,TCCGATATGTACCCGGACGTAATCCGCAGAATAAAGTATCTAAAAG...,1
1,GGTAACGAAGTGGTTACATTATGCTCTTCCGTTTCGGATTGTGAAC...,1
2,GCTTCGTCACGGACGTCCCTCGATTAGACTCGATATGTACCTAACG...,1
3,ATGCTTCTACACGCATTATACAGCCCCGCGTTTTAGTCCTTTTCAG...,1
4,GGACTAGTTGAGGAACAATGAACTCTTTTCAATTGAAACCGAGTTA...,1


In [3]:
df.shape

(60000, 2)

In [4]:
temp_sequences=df[0]
temp_target=df[1]

In [5]:
dict_i = {"A" : 0, "C": 1, "G": 2, "T" : 3}

In [6]:
def one_hot_encode(s):
    i=0
    ohe = np.zeros((len(s), 4))
    for k in s:
        ohe[i,dict_i[k]] = 1
        i+=1
    return ohe

In [7]:
ohe_sequences = np.array([one_hot_encode(s) for s in temp_sequences])
print ohe_sequences
ohe_sequences.shape

[[[0. 0. 0. 1.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]
  ...
  [1. 0. 0. 0.]
  [0. 0. 0. 1.]
  [0. 0. 1. 0.]]

 [[0. 0. 1. 0.]
  [0. 0. 1. 0.]
  [0. 0. 0. 1.]
  ...
  [0. 1. 0. 0.]
  [1. 0. 0. 0.]
  [0. 1. 0. 0.]]

 [[0. 0. 1. 0.]
  [0. 1. 0. 0.]
  [0. 0. 0. 1.]
  ...
  [1. 0. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 1. 0.]]

 ...

 [[0. 0. 0. 1.]
  [0. 0. 1. 0.]
  [0. 1. 0. 0.]
  ...
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [0. 0. 0. 1.]]

 [[1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]
  ...
  [1. 0. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 1. 0.]]

 [[0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]
  ...
  [0. 0. 0. 1.]
  [0. 0. 0. 1.]
  [0. 0. 0. 1.]]]


(60000, 200, 4)

In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Dropout,Flatten
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

We used the Keras deep learning framework (https://github.com/keras-team/keras) to train a CNN to classify
the sequences. The sequences were represented using a one-hot encoding with 4 channels (A, C, G and T). The
CNN architecture is as follows: Layer 1 is a convolutional layer with 40 filters of size 19 and ReLU activation
operating on one-hot encoded input sequences. Layer 2 is a max pooling layer of pool length 10. Layer 3 is a
fully connected layer of size 200 with dropout (p=0.5) and ReLU activation. Layer 4 is a fully connected layer
with a sigmoid activation. The model was trained with the Adam optimizer and binary cross-entropy loss until
no improvement was seen for 3 epochs on the validation set. The datasets, code and model are available at

In [9]:
model = Sequential()
model.add(Conv1D(filters=30, kernel_size=15, strides=1, activation='relu', input_shape=(200,4)))

In [10]:
model.add(MaxPooling1D(pool_size=10, strides=10))

In [11]:
model.add(Flatten())
model.add(Dense(200, activation='relu'))

In [12]:
model.add(Dense(1, activation='sigmoid'))

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 186, 30)           1830      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 18, 30)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 540)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               108200    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 110,231
Trainable params: 110,231
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [15]:
ohe_sequences=shuffle(ohe_sequences)

In [16]:
data_train, data_test, labels_train, labels_test = train_test_split(ohe_sequences, temp_target, test_size=0.30, random_state=42)

In [17]:
model.fit(data_train, labels_train, epochs=50, verbose=1)

Epoch 1/50
42000/42000 [==============================] - 8s 198us/step - loss: 0.6953 - acc: 0.4975
Epoch 2/50
42000/42000 [==============================] - 7s 158us/step - loss: 0.6927 - acc: 0.5130
Epoch 3/50
42000/42000 [==============================] - 7s 162us/step - loss: 0.6905 - acc: 0.5308
Epoch 4/50
42000/42000 [==============================] - 7s 163us/step - loss: 0.6837 - acc: 0.5551
Epoch 5/50
42000/42000 [==============================] - 7s 163us/step - loss: 0.6697 - acc: 0.5896
Epoch 6/50
42000/42000 [==============================] - 7s 165us/step - loss: 0.6379 - acc: 0.6350
Epoch 7/50
42000/42000 [==============================] - 7s 163us/step - loss: 0.5700 - acc: 0.7072
Epoch 8/50
42000/42000 [==============================] - 7s 163us/step - loss: 0.4605 - acc: 0.7937
Epoch 9/50
42000/42000 [==============================] - 7s 161us/step - loss: 0.3454 - acc: 0.8628
Epoch 10/50
42000/42000 [==============================] - 7s 163us/step - loss: 0.2393 - a

In [18]:
scores = model.evaluate(data_test, labels_test)
print ("Test loss ", scores[0])
print ("Test acc ", scores[1])

18000/18000 [==============================] - 1s 65us/step
('Test loss ', 4.100209796057807)
('Test acc ', 0.5005)


In [19]:
model.get_config()

[{'class_name': 'Conv1D',
  'config': {'activation': 'relu',
   'activity_regularizer': None,
   'batch_input_shape': (None, 200, 4),
   'bias_constraint': None,
   'bias_initializer': {'class_name': 'Zeros', 'config': {}},
   'bias_regularizer': None,
   'data_format': 'channels_last',
   'dilation_rate': (1,),
   'dtype': 'float32',
   'filters': 30,
   'kernel_constraint': None,
   'kernel_initializer': {'class_name': 'VarianceScaling',
    'config': {'distribution': 'uniform',
     'mode': 'fan_avg',
     'scale': 1.0,
     'seed': None}},
   'kernel_regularizer': None,
   'kernel_size': (15,),
   'name': 'conv1d_1',
   'padding': 'valid',
   'strides': (1,),
   'trainable': True,
   'use_bias': True}},
 {'class_name': 'MaxPooling1D',
  'config': {'name': 'max_pooling1d_1',
   'padding': 'valid',
   'pool_size': (10,),
   'strides': (10,),
   'trainable': True}},
 {'class_name': 'Flatten',
  'config': {'data_format': 'channels_last',
   'name': 'flatten_1',
   'trainable': True}},


In [ ]:
model.ti